In [ ]:
import os
import torch
import trimesh

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_mesh
from shap_e.util.image_util import load_image
from shap_e.rendering.ply_util import write_ply

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [ ]:
output_folder = "plys"  # 원하는 폴더 이름으로 변경
os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성

output_folder = "plys"  # 원하는 폴더 이름으로 변경
os.makedirs(output_folder, exist_ok=True)  # 폴더가 없으면 생성

In [ ]:
batch_size = 1
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("example_data/corgi.png")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

In [ ]:
mesh = decode_latent_mesh(xm, latents).tri_mesh()
mesh = trimesh.Trimesh(vertices=mesh.verts, faces=mesh.faces)

file_name = f'scale_{guidance_scale}.ply'
file_path = os.path.join(output_folder, file_name)
mesh.export(file_path)

In [ ]:
# render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
# size = 64 # this is the size of the renders; higher values take longer to render.

# cameras = create_pan_cameras(size, device)
# for i, latent in enumerate(latents):
#     images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
#     display(gif_widget(images))